In [5]:
search_for = 121
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004111766815185547
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 15
all cases: 237758115
type: [1, 1, 1, 1, 121] 121
cases of this type: 214358881
100000 15.412293993277128
200000 19.731181274140397
300000 17.72924512286049
400000 17.133603520266366
500000 19.345919481501596
600000 17.456660335285726
700000 16.947118465017677
800000 16.010738401541122
900000 10.499403575816402
1000000 14.557874519421542
1100000 18.16358484940789
1200000 16.24650512787494
1300000 17.201724680550594
1400000 17.761252699878522
1500000 16.295494378269392
1600000 17.920579184194143
1700000 16.975864830164372
1800000 8.787579615202874
1900000 16.20676125891575
2000000 19.505309164252346
2100000 19.241315008188906
2200000 19.831087888678372
2300000 20.40989138407555
2400000 18.77957878786149
2500000 19.859361146024128
2600000 18.416118974698342
2700000 7.9530233029570185
2800000 19.070950479700386
2900000 19.555689882005836
30000

29400000 13.982386676575462
29500000 14.412719941455043
29600000 14.868423492295891
29700000 16.243198620548743
29800000 15.922971969823868
29900000 8.824195383376738
30000000 16.029821996813087
30100000 15.456226219212569
30200000 15.666639825392688
30300000 16.134958073066006
30400000 8.284844226587115
30500000 16.240889039023926
30600000 15.249669692634384
30700000 14.631248809021217
30800000 15.123647324761064
30900000 13.722602410452186
31000000 10.278256912017975
31100000 10.268271602509108
31200000 14.009153980556011
31300000 14.874162036085412
31400000 14.826722014200287
31500000 16.03065970514549
31600000 15.480830175726547
31700000 8.655122962514973
31800000 15.62432803608653
31900000 15.415365944483385
32000000 15.146125708424442
32100000 15.521251493206213
32200000 7.5870346046139066
32300000 15.990403424727583
32400000 14.730115994521023
32500000 14.691983161008743
32600000 13.437678226465145
32700000 13.704768925663767
32800000 6.577289221035919
32900000 13.24242080875678

59000000 6.076033734268596
59100000 11.808879353833506
59200000 12.006082292126438
59300000 12.799598694615867
59400000 5.281888391994317
59500000 12.330153981249495
59600000 12.327547939518558
59700000 12.331933679366202
59800000 7.135616155128883
59900000 12.941732894223923
60000000 12.542699064681402
60100000 13.347840401952238
60200000 13.248790912325267
60300000 13.232464976891459
60400000 13.104403100117823
60500000 13.142629284960714
60600000 12.11660432798174
60700000 12.241499022641253
60800000 5.357459517368555
60900000 11.206036396711578
61000000 11.661164624098337
61100000 9.93158005653707
61200000 6.852020871263454
61300000 11.513143263391859
61400000 11.736573426686613
61500000 8.620952951524766
61600000 10.411485932880627
61700000 11.925546836369499
61800000 11.930552993868975
61900000 13.043105344547076
62000000 12.63953123643224
62100000 13.2039575431317
62200000 13.705332460902351
62300000 12.896855896855069
62400000 12.535099651000946
62500000 11.05673940861445
62600

88800000 10.62772551254325
88900000 10.045691767987977
89000000 9.67187239598903
89100000 9.719231585460792
89200000 9.83824653939611
89300000 9.054383612009568
89400000 6.509361574432164
89500000 3.976345682540794
89600000 8.836664761729086
89700000 7.372797467366824
89800000 9.92255403690322
89900000 9.510537616539422
90000000 9.960013662419358
90100000 9.628609560891146
90200000 10.370564462175205
90300000 11.272390000833452
90400000 10.970896487434423
90500000 10.691505736839762
90600000 10.489309385052463
90700000 9.958497873615142
90800000 9.6762558229351
90900000 10.217315829920192
91000000 9.753212912308143
91100000 7.275466292922087
91200000 7.219032816177636
91300000 4.423318872937153
91400000 6.001144772378478
91500000 9.850985041205554
91600000 10.313598092845675
91700000 9.44671949555491
91800000 10.545459691904394
91900000 10.218190899599426
92000000 10.423665304259691
92100000 11.185438412738083
92200000 11.242266421985999
92300000 10.647090595956008
92400000 10.16072584

118100000 7.699591782062177
118200000 8.096605872345984
118300000 7.927200096939278
118400000 7.94040021658161
118500000 7.96080322798439
118600000 8.229071785844349
118700000 8.608140192577862
118800000 8.577034850632936
118900000 8.135178155823915
119000000 7.89120042027242
119100000 7.955611061460794
119200000 7.987456206160021
119300000 7.842915710189669
119400000 7.634609029937516
119500000 4.9681880402286644
119600000 3.8051298354700145
119700000 3.9865758963461517
119800000 6.516408822786411
119900000 7.65218333253068
120000000 7.608759585187257
120100000 8.010722633426662
120200000 7.813204765712247
120300000 8.221871958390276
120400000 8.582667441104885
120500000 8.818221761039995
120600000 8.631792184818098
120700000 7.414762843548235
120800000 7.841693439414584
120900000 7.396111998105764
121000000 7.23546266725069
121100000 7.694039234460618
121200000 7.307740076403141
121300000 4.620322149977482
121400000 3.7998033252262475
121500000 4.8726775686885615
121600000 6.85768809

147400000 6.5593632492885625
147500000 6.373978201393568
147600000 4.155896143213988
147700000 5.632613306972541
147800000 5.918650571966168
147900000 5.852075660543293
148000000 3.535513986926576
148100000 6.3154068473417855
148200000 5.907470818099907
148300000 3.1910779880615374
148400000 6.060451347262462
148500000 6.222565277776112
148600000 7.054366273444997
148700000 6.832600274608006
148800000 6.597316180386007
148900000 6.662011221003986
149000000 6.7706562167861195
149100000 6.265183162223488
149200000 6.459502797741215
149300000 6.024947395083265
149400000 3.4661733657972316
149500000 6.023786308007806
149600000 5.815294076056242
149700000 3.916901055634976
149800000 4.529986621746252
149900000 6.279201413606432
150000000 5.742032419721662
150100000 2.9150751641541093
150200000 5.805381719598644
150300000 6.00230012496519
150400000 6.759298629787406
150500000 6.749672806936109
150600000 6.7119107088501755
150700000 6.627042369214528
150800000 6.678480790062921
150900000 6.40

176600000 4.263145376152293
176700000 4.306994486258447
176800000 4.520259526571738
176900000 2.137433031574756
177000000 4.51609061687697
177100000 4.544289124021007
177200000 4.419837517378665
177300000 4.54949556163902
177400000 4.183018429033287
177500000 2.5361825323971345
177600000 4.2623343289529085
177700000 4.251354906617291
177800000 4.0816488323770495
177900000 3.745823397301614
178000000 4.012039711330698
178100000 1.833687263597942
178200000 3.7334306272147697
178300000 4.010386351466745
178400000 3.9221736093028485
178500000 4.0864191486961845
178600000 4.309172880728794
178700000 2.5419957819880676
178800000 4.224443768814663
178900000 4.511576308551854
179000000 4.3411717405475345
179100000 4.2446435986100335
179200000 3.7336759107711615
179300000 3.071637233674387
179400000 4.15436016588822
179500000 4.021844740717776
179600000 4.080744428455833
179700000 3.9789670067661103
179800000 3.1821458167306083
179900000 2.4875905271638468
180000000 3.892086234930337
180100000 

205400000 2.237619632034726
205500000 1.470184597458333
205600000 1.348111254421105
205700000 2.5481659321642622
205800000 2.535543265264501
205900000 2.504134787475338
206000000 2.495618235316912
206100000 2.266784773689707
206200000 2.2992465690307387
206300000 2.4066326506386964
206400000 1.363559939956218
206500000 2.0484454686215354
206600000 2.3807821841080523
206700000 2.2539992213710844
206800000 2.3256243239798415
206900000 2.532756991919524
207000000 2.3979649511565464
207100000 2.268907315858907
207200000 1.9795477739605936
207300000 1.5205929535970688
207400000 1.4272640391106937
207500000 2.361564180272059
207600000 2.298200010909041
207700000 2.3131722224666342
207800000 2.2341973659835426
207900000 2.2840432055364452
208000000 2.105497082363635
208100000 2.2223332025444176
208200000 0.8884365563684107
208300000 2.362174006697499
208400000 2.139662561824196
208500000 2.1153228345499238
208600000 2.220059861591094
208700000 2.177748597569459
208800000 2.2574014021369315
20

233300000 0.10121063874969548
233400000 0.13971925028577448
233500000 0.14128651618842283
233600000 0.15642207939562533
233700000 0.1455658712929984
233800000 0.13023243333640697
best so far: 0
type: [1, 1, 1, 121, 1] 121
cases of this type: 1771561
233900000 0.0811930107686288
234000000 0.026702846229364478
234100000 0.024076237002988655
234200000 0.024306976080119606
234300000 0.02309154370763898
234400000 0.02257502792159716
234500000 0.02451119264111254
234600000 0.022661733499636252
234700000 0.021220170538905594
234800000 0.02082165995266371
234900000 0.020160366477499407
235000000 0.018565453515860766
235100000 0.018009080402258373
235200000 0.01720190029710531
235300000 0.016752806757767993
235400000 0.016036226489394904
235500000 0.015206150105416776
235600000 0.015052341386410926
best so far: 0
type: [1, 1, 11, 1, 11] 121
cases of this type: 1771561
235700000 0.057463046002225744
235800000 0.06278828697119157
235900000 0.054278105966604415
236000000 0.055508976923796874
23610